In [ ]:
import os
import pandas as pd
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html

In [ ]:
TPCDS_FN = "tpcds_queries_df.csv"
JOB_FN = "job_queries_df.csv"
CEB_FN = "ceb_queries_df.csv"

In [ ]:
tpdf = pd.read_csv(TPCDS_FN)
jobdf = pd.read_csv(JOB_FN)
cebdf = pd.read_csv(CEB_FN)
tpdf["dataset"] = "TPCDS"
jobdf["dataset"] = "JOB"
cebdf["dataset"] = "CEB"

In [ ]:
print(len(tpdf))
print(len(jobdf))
print(len(cebdf))

In [ ]:
df = pd.concat([tpdf, jobdf, cebdf])

In [ ]:
print(df.keys())

In [ ]:
# HTML(df[["like_ops", "discrete_ops", "cont_ops"]].\
#      describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

In [ ]:
def pretty_print(df):
    return display( HTML( df.to_html(index=False).replace("\\n","<br>") ) )

In [ ]:
STATCOLS = ["like_ops", "discrete_ops", "cont_ops"]
STATSTODROP = ["std", "min", "max", "25%", "50%","75%"]
curdrops = []

for si, stc in enumerate(STATCOLS):
    for sdrop in STATSTODROP:
        curdrops.append(tuple([stc, sdrop]))
    if si != 0:
        curdrops.append(tuple([stc,"count"]))

#cdf = df[[""]]
cdf = df[["dataset","like_ops", "cont_ops", "discrete_ops", "in_ops"]]

pretty_print(cdf.groupby(["dataset"])[STATCOLS].describe().\
     reset_index().\
     drop(columns=curdrops).\
     rename(columns={"like_ops":"LIKE", "discrete_ops":"Discrete", "cont_ops":"Continuous",
                    "mean":"Fraction"}).\
     round(3))

In [ ]:
discdf = df[df.discrete_ops == 1]

tmp = discdf.groupby(["dataset","dkind"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

# pretty_print(tmp.reset_index()\
#              .rename(columns={"discrete_ops":"Percentage"})
#             )
tmp = tmp.reset_index().set_index(['dataset', 'dkind']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

In [ ]:
tmp = discdf.groupby(["dataset","dtype"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

# pretty_print(tmp.reset_index()\
#              .rename(columns={"discrete_ops":"Percentage"})
#             )

tmp = tmp.reset_index().set_index(['dataset', 'dtype']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

In [ ]:
# tmp = discdf.groupby(["dataset","dtype"]).count()["discrete_ops"]
# tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
# tmp.reset_index()

In [ ]:
#ldf["val"]
def get_len(val):
    val = val.replace("%", "")
    val = val.replace("'", "")
    return len(val)

In [ ]:
ldf = df[df.like_ops == 1]
#ldf.groupby("likekind").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

ldf["Filter Length"] = ldf.apply(lambda x: get_len(x["val"]), axis=1)
tmp = ldf.groupby(["dataset","likekind"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))


tmp = tmp.reset_index().set_index(['dataset', 'likekind']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

In [ ]:
tmp = ldf.groupby(["dataset","likekind"])[["Filter Length"]].describe(percentiles=[0.9])
#tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
DROPS = ["std", "min", "max"]
dc = []
for dr in DROPS:
    dc.append(("Filter Length", dr))

tmp = tmp.reset_index().set_index(['dataset', 'likekind']).rename(
    columns={}).drop(columns=dc)
tmp

In [ ]:
#ldf = df[df.like_ops == 1]

#ldf["likedtype"] = ldf.apply(lambda x: if x["likedtype"] == -1: then ,axis=1)

ldtypes = []
for idx,row in ldf.iterrows():
    if row["likedtype"] == "-1" or row["likedtype"] == -1 or row["likedtype"] == "-1.0":
        ldtypes.append("unknown")
    else:
        ldtypes.append(row["likedtype"])
ldf["likedtype"] = ldtypes

tmp = ldf.groupby(["dataset","likedtype"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
tmp = tmp.reset_index().set_index(['dataset', 'likedtype']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

In [ ]:
#inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

In [ ]:
allinpdata = defaultdict(list)
for idx, row in df.iterrows():
    op = row["op"]
    if op in ["=", "!="]:
        allinpdata["col"].append(row["col"])
        allinpdata["val"].append(row["val"])
        allinpdata["dataset"].append(row["dataset"])

    elif op == "in":
        if "select" in row["val"]:
            pass
        else:
            cvals = row["val"]
            cvals = cvals.replace("(", "")
            cvals = cvals.replace(")", "")
            cvals = cvals.split(",")
            for cval in cvals:
                allinpdata["dataset"].append(row["dataset"])
                allinpdata["col"].append(row["col"])
                allinpdata["val"].append(cval)

In [ ]:
inpdata = pd.DataFrame(allinpdata)
print(len(inpdata))
inpdata = inpdata.drop_duplicates()
inpdata = inpdata[inpdata["col"] != "function"]
print(len(inpdata))
print(len(set(inpdata["col"])))
print(set(inpdata["col"]))

In [ ]:
inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

In [ ]:
STATCOLS = ["like_ops", "discrete_ops", "cont_ops"]
STATSTODROP = ["std", "min", "max", "25%", "50%","75%"]
curdrops = []

pretty_print(
    inpdata.groupby(["dataset", "col"]).nunique().reset_index()[["dataset","val"]]\
    .groupby(["dataset"])[["val"]].describe(percentiles=[0.75,0.9,0.99]).reset_index()\
    .rename(columns={"count":"Total \\n Columns", "val":"Number of unique constants"})\
    .round(3)
)